In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix


In [ ]:
X = np.genfromtxt('Datasets/RRLyrae_features.txt',delimiter=',')
y = np.genfromtxt('Datasets/RRLyrae_labels.txt')

In [ ]:
#Let's take a look at the data set to see if it's balanced or not

In [ ]:
plt.hist(y);

In [5]:
np.sum(y > 0.5), np.sum(y < 0.5) #trick to count objects with label 1/0

(483, 92658)

In [6]:
from sklearn.svm import SVC #Support Vector Classifier

In [7]:
model = SVC() # What is this doing to the parameters?

In [8]:
model

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Let's start with one single train/test split.

In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, \
                                                test_size = 0.2, \
                                                random_state=0) 


model.fit(Xtrain,ytrain)

ypred = model.predict(Xtest)

In [10]:
metrics.accuracy_score(ytest,ypred)

0.99559826077620917

In [11]:
metrics.recall_score(ytest,ypred)

0.0

### What is happening?

To judge the extent of our problem we can use 5-fold cross validation. This will show us if the problem is relative to this particular train/test split or not.

As a reminder, cross_val_score picks k (in our case, 5) different disjoint folds, using one fold at a time as the test set, and the other k-1 (in our case, 4) as training set, and averages the test scores.

In [12]:
scores = cross_val_score(model, X, y, cv=5, scoring= 'accuracy') # What does this do?

In [13]:
scores

array([ 0.99479306,  0.99479306,  0.99479306,  0.99484619,  0.99484619])

In [14]:
scores = cross_val_score(model, X, y, cv=5, scoring= 'recall') # What does this do?

In [15]:
scores

array([ 0.,  0.,  0.,  0.,  0.])

Our data set is so imbalanced that SVCs are creating a classifier that says everything is negative! To proceed more quickly, let's work with a smaller data set that has also been artificially balanced by me.

In [16]:
Xlittle = np.genfromtxt('RRLyrae_features_small.txt',delimiter=',')
ylittle = np.genfromtxt('RRLyrae_labels_small.txt')

In [17]:
ylittle.shape

(2483,)

In [18]:
np.sum(ylittle > 0.5), np.sum(ylittle < 0.5)

(483, 2000)

In [19]:
Xtrain, Xtest, ytrain, ytest = train_test_split(Xlittle, ylittle, \
                                                test_size = 0.2, \
                                                random_state=2) 

model.fit(Xtrain,ytrain)

ypred = model.predict(Xtest)

print(metrics.recall_score(ypred,ytest))

print(metrics.accuracy_score(ypred,ytest))

print(metrics.precision_score(ypred,ytest))


0.913793103448
0.969818913481
0.954954954955


### 5-fold cross validation.

Note that we are using a built-in cross validation function called Stratified KFold. This means that the folds that will be created will approximately maintain the same class distribution of the original data set. This is useful in an imbalanced data set. The "shuffle = True" option ensures that we shuffle the data set before proceeding, useful if the classes are in order (e.g. all the zeros in the beginning, all the ones at the end).

In [20]:
ylittle

array([ 0.,  0.,  0., ...,  1.,  1.,  1.])

In [22]:
model = SVC() 
cvmethod = StratifiedKFold(shuffle=True, n_splits=5) # this ensures that folds are randomized
scores = cross_val_score(model, Xlittle, ylittle, cv=cvmethod, \
                         scoring= 'recall') # What does this do?
print(scores) 
print(np.mean(scores))
print(np.std(scores))

[ 0.97938144  0.97938144  1.          0.9375      0.95833333]
0.970919243986
0.0212798450198


The stuff above gives us an idea of the SVC performance. Let's compare it to our old friend the decision tree.

In [23]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier() #Leaving params at default value for now

scores = cross_val_score(model, Xlittle, ylittle, cv=cvmethod, \
                         scoring= 'recall') # What does this do?

print(scores) 
print(np.mean(scores))
print(np.std(scores))

[ 0.93814433  0.91752577  0.94845361  0.90625     0.9375    ]
0.929574742268
0.0153745268577


It looks like SVC is superior to Decision Trees even with no parameter optimization. But let's see if we can do better with nested cross validation.

### Parameter optimization

We can do one parameter search on our own just to see how it works and then use sklearn's built-in function.

Let's say that we want to optimize the parameter C, the soft margin. Possible values are C = [1, 10, 100].

In [ ]:
#Important! COMMENT

#Outer k-fold:
    
outercv = StratifiedKFold(n_splits=5, shuffle=True) #creates 5 disjoint splits

innercv = StratifiedKFold(n_splits=4, shuffle=True) #creates 4 disjoint splits

C = [1,10,100]

i=0

winning_model_scores = []

for train_index, test_index in outercv.split(Xlittle,ylittle): #This runs the outer cross validation
    
    i+=1
    
    print('Fold ' ,i, 'outer cross validation')
    
    Xlittle_train = Xlittle[train_index] #This is my "yellow" training set where I do the inner CV
    ylittle_train = ylittle[train_index]
    
    Xlittle_test = Xlittle[test_index]
    ylittle_test = ylittle[test_index]
    
    temp_scores = []
    
    for Cvalue in C:
        
        model = SVC(C=Cvalue)

        scores = cross_val_score(model, Xlittle_train, ylittle_train, \
                                 cv=innercv, scoring= 'recall') 
        
        print('C =', Cvalue, 'Mean score over 4 inner folds =', \
              "{:.4f}".format(np.mean(scores)))
    
        temp_scores.append(np.mean(scores))
    
    print('Winning model has C =', C[np.argmax(temp_scores)]), \
        'and score = ', temp_scores[np.argmax(temp_scores)]

    # Now we set the parameters of the SVC to the best C we just found, fit the model 
    # and predict labels of of test set for each outer (blue) fold
    
    model = SVC(C = C[np.argmax(temp_scores)])
    
    model.fit(Xlittle_train, ylittle_train)
    
    ypred = model.predict(Xlittle_test)
    
    winning_model_scores.append(metrics.recall_score(ylittle_test,ypred)) #append this to the outer cv results
    
print('The average of the winning model scores \
    (i.e. the generalization error) is', \
      "{:.4f}".format(np.mean(winning_model_scores)), \
      "{:.4f}".format(np.std(winning_model_scores) ))


You don't actually need to do this by hand!

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
import time #This is to see how long things take

In [ ]:
#optimizing SVC: this replaces the inner loop!
#Important! 

parameters = {'kernel':['rbf','poly'], \
              'gamma':[0.01, 0.1, 0.5], 'C':[1,10,100]}
nmodels = np.product([len(el) for el in parameters.values()])
start = time.time()
model = GridSearchCV(SVC(), parameters, cv = innercv, scoring = 'recall', \
                     verbose = 1, n_jobs = 4)
model.fit(Xlittle_train, ylittle_train)
stop = time.time()
print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)
print('Time per model (s):', "{:.4f}".format((stop-start)/float(nmodels*4)))

### Now putting everything together!

In [ ]:
#Important! COMMENT

#Outer k-fold:
    
outercv = StratifiedKFold(n_splits=5, shuffle=True) #creates 5 disjoint splits

innercv = StratifiedKFold(n_splits=4, shuffle=True) #creates 4 disjoint splits

i=0

winning_model_scores = []

for train_index, test_index in outercv.split(Xlittle,ylittle): #This runs the outer cross validation
    
    i+=1
    
    print('Fold ' ,i, 'outer cross validation')
    
    Xlittle_train = Xlittle[train_index] #"yellow" training set
    ylittle_train = ylittle[train_index]
    
    Xlittle_test = Xlittle[test_index]
    ylittle_test = ylittle[test_index]
    
    #optimizing SVC: this replaces the inner loop!
    
    parameters = {'kernel':['rbf','poly'], \
              'gamma':[0.01, 0.1, 0.5], 'C':[1,10,100]}
    nmodels = np.product([len(el) for el in parameters.values()])
    start = time.time()
    model = GridSearchCV(SVC(), parameters, cv = innercv, scoring = 'recall', \
                     verbose = 1, n_jobs = 4)
    model.fit(Xlittle_train, ylittle_train)
    stop = time.time()
    print('Best params, best score:', "{:.4f}".format(model.best_score_), model.best_params_)
    print('Time per model (s):', "{:.4f}".format((stop-start)/float(nmodels*4)))

    #Compute test scores with optimal parameters on outer i-th test fold
    
    winner = model.best_estimator_
    
    winner.fit(Xlittle_train, ylittle_train)
    
    ypred = winner.predict(Xlittle_test)
    
    winning_model_scores.append(metrics.recall_score(ylittle_test,ypred)) #append this to the outer cv results
    
print('The average of the winning model scores (i.e. the generalization error) is', \
      np.mean(winning_model_scores), np.std(winning_model_scores) )


### Conclusion: The performance that you can expect from your classifier is recall = .....

### Advanced bit: Your final model will be built by doing grid-search CV on the full data set, and its anticipated generalization error will be the one given above.

In [ ]:
#Note: this might have different params

parameters = {'kernel':['rbf','poly'], \
              'gamma':[0.01, 0.1, 0.5], 'C':[1,10,100]}
nmodels = np.product([len(el) for el in parameters.values()])
start = time.time()
model = GridSearchCV(SVC(), parameters, cv = innercv, scoring = 'recall', \
                     verbose = 1, n_jobs = 4)
model.fit(Xlittle, ylittle)
stop = time.time()
print('Best params, best score:', "{:.4f}".format(model.best_score_), model.best_params_)
print('Time per model (s):', "{:.4f}".format((stop-start)/float(nmodels*4)))

#Final model

model.best_estimator_.fit(Xlittle,ylittle)

### Exercise

Do Grid Search CV on the full data set, varying for time's sake only gamma and the class weight, and setting the number of folds to be 3 in both inner and outer loops. In other words

parameters = {'kernel':['rbf'],'C':[10], 'gamma':[0.01, 0.1, 0.5], 'class_weight':[{1:1},{1:3},{1:5},{1:7}]}

Note: It might take some time (on my laptop it took ~ 10 minutes); set verbose = 2 if you want to follow the progress. 